## Input problem data and imported libs

In [61]:
import sympy as sp
import numpy as np
import functools as fnt
import itertools as itt


t = sp.symbols('t')
A = sp.Matrix([[-2, 8, 4], [1, -4, -2], [-3, 12, 6]])
B = sp.Matrix([1, 1, 1])
F = sp.Matrix([-15*t, 0, -20*t])
x0 = sp.Matrix([1, 3, 0])
x1 = sp.Matrix([24, -7, 34])
T = 1

# A = sp.Matrix([[0, 2], [-2, 0]])
# B = sp.Matrix([1, 2])
# F = sp.Matrix([0, 0])
# x0 = sp.Matrix([5, 0])
# x1 = sp.Matrix([10, 5])
# T = sp.pi

# A = sp.Matrix([[2, -16, -6], [1, -8, -3] , [-2, 16, 6]])
# B = sp.Matrix([1, 0, -2])
# F = sp.Matrix([0, -7*t, -14*t])
# x0 = sp.Matrix([2, 0, -1])
# x1 = sp.Matrix([13, 5, -13])
# T = 1

dim = A.shape[0]

print(f'A: {A}')
print(f'B: {B}')

A: Matrix([[-2, 8, 4], [1, -4, -2], [-3, 12, 6]])
B: Matrix([[1], [1], [1]])


## Matrix A eigenvalues

In [62]:
theta = sp.symbols('theta')
chpoly = A.charpoly(theta).coeffs()
print(chpoly)
eigs = A.eigenvals() 
print(eigs)

[1]
{0: 3}


## Fundamental matrix calculating

In [63]:
eqs = []
omega = sp.symbols('omega')
b = sp.symbols('b0:%d'%dim)
def lamda_power_list(dim) : return np.array([omega ** (i) for i in range(dim)])

for eig, mult in eigs.items():
    lpl = lamda_power_list(dim)
    # print(lpl)
    eq = sp.exp(omega * t) - np.dot(b, lpl)
    eqs.append(eq)
    if mult > 1:
        for i in range(1, mult):
            # lpl = lamda_power_list(lamda, dim, 0)
            # eq = (t ** i) * sp.E ** (lamda * t) - np.dot(b[i:], lpl) 
            eqs.append(sp.diff(eqs[-1], omega)) 

    for i in range(dim):
        eqs[-i] = eqs[-i].subs({omega :eig})

print(f'system: \n{eqs}\n')

b_dict = sp.solve(eqs, b)
print(f'solution: \n{b_dict}\n')
b_val = sp.zeros(rows=1, cols=dim)
for i in range(dim):
    b_val[i] = b_dict[b[i]]

Y = sp.zeros(dim, dim)
for i in range(dim):
    Y += (A ** i) * b_val[i]
print(f'Fundamental matrix: \n{repr(Y)}')


system: 
[1 - b0, -b1 + t, -2*b2 + t**2]

solution: 
{b0: 1, b1: t, b2: t**2/2}

Fundamental matrix: 
Matrix([
[1 - 2*t,     8*t,     4*t],
[      t, 1 - 4*t,    -2*t],
[   -3*t,    12*t, 6*t + 1]])


## Calculating vector c for program control

In [64]:
Y_inv = Y.inv()
print(Y_inv)
Q = Y_inv * B
print(f'Q: \n{repr(Q)}\n')
P = sp.integrate(Q * sp.transpose(Q), (t, 0, T))
print(f'P: \n{repr(P)}\n')
N = Y_inv.subs({t : 0}) * x1 - x0 - sp.integrate(Y_inv * F, (t, 0, T))
print(f'N: \n{repr(N)}\n')
print(sp.det(P))
# c = P.inv() * N
c = sp.symbols('c0:%d'%dim)
C = sp.Matrix(c)
print(sp.solve(P * C - N))
# print(c)

Matrix([[2*t + 1, -8*t, -4*t], [-t, 4*t + 1, 2*t], [3*t, -12*t, 1 - 6*t]])
Q: 
Matrix([
[1 - 10*t],
[ 5*t + 1],
[1 - 15*t]])

P: 
Matrix([
[  73/3, -109/6, 77/2],
[-109/6,   43/3,  -29],
[  77/2,    -29,   61]])

N: 
Matrix([
[83/6],
[-5/3],
[  19]])

0
{c0: 224/25 - 4*c2/3, c1: c2/3 + 281/25}
